In [ ]:
import pandas as pd
import requests
import json

In [ ]:
API_URL = "https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003"
API_GATEWAY_KEY = "YOUR_KEY"
API_KEY = "YOUR_KEY"
REQUEST_ID = "YOUR_KEY"
API_HOST = "https://clovastudio.stream.ntruss.com"

In [ ]:
#excel 파일명 list
excel_fname = ["your dataset"]

In [ ]:
instruction = """
"너는 전문적인 의사이자 약사야. 내가 의약품에 관한 질문을 하면 아래 답변 규칙에 따라서 친절하고 전문적으로 대답해.\n
###답변 규칙###\n
1. 답변에서 면책문구를 모두 제거해.\n
2. 인간에게 상담 혹은 조언을 받으라는 문구 모두 제거해\n
3. 상황에 따라 답변이 달라지면 각각의 상황 별로 답변해\n
4. 질문에만 대답해.\n
###질의응답 예시###\n
질문 : Q?\n
답변 : A\n
질문 : Q\n
답변 : A"""

In [ ]:
# API 호출 함수
def call_clova_api(question):
    retry_cnt = 0
    while retry_cnt < 5:
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': API_KEY,
            'X-NCP-APIGW-API-KEY': API_GATEWAY_KEY,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': REQUEST_ID,
            'Content-Type': 'application/json; charset=utf-8'
        }
        request_data = {
            'messages': [{"role" : "system", "content": instruction}
                         , {"role" : "user","content" : question}],
            'topP': 0.8,
            'topK': 0,
            'maxTokens': 256,
            'temperature': 0.5,
            'repeatPenalty': 5.0,
            'stopBefore': [],
            'includeAiFilters': False,
            'seed': 0
        }
        try:
            response = requests.post(API_HOST + '/testapp/v1/chat-completions/HCX-003', headers=headers, json=request_data)
            response = response.json()
            return response['result']['message']['content']
        except Exception as e:
            retry_cnt = retry_cnt + 1
            return "Failed to get an answer. ATTEMPT : " + str(retry_cnt)

In [ ]:
#파일마다 api를 호출해서 답변을 받는다.
for fname in excel_fname:
    #엑셀 파일 pandas dataframe 변환
    df = pd.read_excel(fname)
    #question list에 질문 저장
    questions = df['question']
    answers = []
    num = 0
    #질문에 대한 답변을 받아서 answers list에 저장.
    for question in questions:
        answer = call_clova_api(question)
        answers.append(answer)
        print("question ", num," : ",question,"\n answer : ",answer)
        num = num+1
    #pandas dataframe에 gpt 답변 추가
    df['clovax answer'] = answers
    
    #dataframe 엑셀 변환
    df.to_excel(fname, index = False)